In [ ]:
import pennylane as pl
import numpy as np
import time, random
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

In [ ]:
def mat_1d_Dirichlet(M):
    A = 2*np.identity(M-1)
    for i in range(M-2):
        A[i][i+1] = -1
        A[i+1][i] = -1
    return A

In [ ]:
n_q = 4
m = 2**n_q

mat_D = mat_1d_Dirichlet(m+1)
rhs = np.array([1 for i in range(2**(n_q-1))] + [-1 for i in range(2**(n_q-1))])
normalized_rhs = rhs/np.linalg.norm(rhs)

In [ ]:
sol = np.linalg.solve(mat_D, normalized_rhs)
exact_norm = np.linalg.norm(sol)
normalized_sol = sol/exact_norm

In [ ]:
### get ansatz state ###
dev_get_state = pl.device("default.qubit", wires=n_q)
@pl.qnode(dev_get_state)
def get_ansatz_state(theta):
    ## ansatz unitary
    p = int((len(theta)-n_q)/(2*(n_q-1)))  # depth of ansatz
    for i in range(n_q):
        pl.RY(theta[i], wires=i)
    # pl.Barrier(wires=[i for i in range(n_q)])
    for iter in range(p):
        for i in range(n_q//2):
            pl.CZ(wires=[2*i, 2*i+1])
            pl.RY(theta[n_q+2*iter*(n_q-1)+2*i], wires=2*i)
            pl.RY(theta[n_q+2*iter*(n_q-1)+2*i+1], wires=2*i+1)
        if n_q != 2:
            for i in range((n_q-1)//2):
                pl.CZ(wires=[2*i+1, 2*i+2])
                pl.RY(theta[n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i], wires=2*i+1)
                pl.RY(theta[n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i+1], wires=2*i+2)
        # pl.Barrier(wires=[i for i in range(n_q)])
    return pl.state()

In [ ]:
A_oper_list = []    # Dirichlet
A_oper_list.append('0'*n_q)
for i in range(n_q):
    base_0 = '0'*(n_q-i-1)
    oper_simple = '1'*(i+1)
    A_oper_list.append(base_0+oper_simple)

print(A_oper_list)

In [ ]:
n_shots = 10**4

dev_denom = pl.device("default.qubit", wires=n_q, shots=n_shots)
@pl.qnode(dev_denom)
def qc_denominator_oper(A_oper, theta):
    ## ansatz unitary
    p = int((len(theta)-n_q)/(2*(n_q-1)))  # depth of ansatz
    for i in range(n_q):
        pl.RY(theta[i], wires=i)
    for iter in range(p):
        for i in range(n_q//2):
            pl.CZ(wires=[2*i, 2*i+1])
            pl.RY(theta[n_q+2*iter*(n_q-1)+2*i], wires=2*i)
            pl.RY(theta[n_q+2*iter*(n_q-1)+2*i+1], wires=2*i+1)
        if n_q != 2:
            for i in range((n_q-1)//2):
                pl.CZ(wires=[2*i+1, 2*i+2])
                pl.RY(theta[n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i], wires=2*i+1)
                pl.RY(theta[n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i+1], wires=2*i+2)
    pl.Barrier(wires=[i for i in range(n_q)])
    ## measurement
    if A_oper == '0'*n_q:
        return pl.counts(wires=[i for i in range(n_q)])
    else:
        n_cr = len(A_oper) - A_oper.count('0')
        for i in range(1, n_cr):
            pl.CNOT(wires=[n_q-i-1, n_q-i])
        pl.Hadamard(wires=n_q-n_cr)
        measure_list = [n_q-n_cr+i for i in range(n_cr)]
        return pl.counts(wires=measure_list)

In [ ]:
p = 1
num_param = n_q + 2*p*(n_q-1)
list_param = np.array([i for i in range(16)])*np.pi/8
test_theta = np.array([random.choice(list_param) for i in range(num_param)])
for i in range(n_q):
    fig, ax = pl.draw_mpl(qc_denominator_oper)(A_oper_list[i+1], test_theta)
    plt.show()

In [ ]:
def expectation_denominator_oper(A_oper, counts):
    if A_oper == '0' * n_q:
        expectation = 2
    elif A_oper.count('1') == 1:
        expectation = (counts.get('0', 0) - counts.get('1', 0))/n_shots
        expectation = -expectation
    else:
        expectation = (counts.get('01'+'0'*(A_oper.count('1')-2), 0) - counts.get('11'+'0'*(A_oper.count('1')-2), 0))/n_shots
        expectation = -expectation
    return expectation

In [ ]:
p = 5
num_param = n_q + 2*p*(n_q-1)
list_param = np.array([i for i in range(16)])*np.pi/8
test_theta = np.array([random.choice(list_param) for i in range(num_param)])
state = np.array(get_ansatz_state(test_theta))
test_mat = mat_D
exact_result = np.dot(state, test_mat @ state.transpose())
print('exact_result:', exact_result)
qc_result = 0
for item in A_oper_list:
    results = qc_denominator_oper(A_oper=item, theta=test_theta)
    result = expectation_denominator_oper(A_oper=item, counts=results)
    qc_result += result
print('qc_result:', qc_result)

In [ ]:
def cost_denominator(theta):
    def cost_denominator_oper(A_oper):
        results = qc_denominator_oper(A_oper, theta)
        cost = expectation_denominator_oper(A_oper, counts=results)
        return cost
    denom_cost_list = Parallel(n_jobs=-1, prefer='processes')(delayed(cost_denominator_oper)(A_oper) for A_oper in A_oper_list)
    denom_cost = sum(denom_cost_list)
    return denom_cost

In [ ]:
p = 5
num_param = n_q + 2*p*(n_q-1)
list_param = np.array([i for i in range(16)])*np.pi/8
test_theta = np.array([random.choice(list_param) for i in range(num_param)])
state = np.array(get_ansatz_state(test_theta))
exact_result = np.dot(state, mat_D @ state.transpose())
print('exact_result:', exact_result)
qc_result = cost_denominator(test_theta)
print('qc_result:', qc_result)

In [ ]:
### quantum circuit for estimating <f,psi(theta)|XI^n|f,psi(theta)> ###
dev_numer = pl.device("default.qubit", wires=n_q, shots=n_shots)
@pl.qnode(dev_numer)
def qc_numerator(theta):
    ## ansatz unitary
    p = int((len(theta)-n_q)/(2*(n_q-1)))  # depth of ansatz
    for i in range(n_q):
        pl.RY(theta[i], wires=i)
    for iter in range(p):
        for i in range(n_q//2):
            pl.CZ(wires=[2*i, 2*i+1])
            pl.RY(theta[n_q+2*iter*(n_q-1)+2*i], wires=2*i)
            pl.RY(theta[n_q+2*iter*(n_q-1)+2*i+1], wires=2*i+1)
        if n_q != 2:
            for i in range((n_q-1)//2):
                pl.CZ(wires=[2*i+1, 2*i+2])
                pl.RY(theta[n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i], wires=2*i+1)
                pl.RY(theta[n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i+1], wires=2*i+2)
    ## measurement
    pl.PauliZ(wires=0)
    for i in range(n_q):
        pl.Hadamard(wires=i)
    return pl.counts(wires=[i for i in range(n_q)])

### estimate numerator ###
def cost_numerator(theta):
    counts = qc_numerator(theta)
    numer_cost = counts.get('0'*n_q, 0)/n_shots
    return numer_cost

In [ ]:
p = 5
num_param = n_q + n_q + 2*p*(n_q-1)
list_param = np.array([i for i in range(16)])*np.pi/8
test_theta = np.array([random.choice(list_param) for i in range(num_param)])
ansatz_state = np.array(get_ansatz_state(test_theta))
exact_result = np.dot(ansatz_state, normalized_rhs)
exact_result = exact_result**2
print('exact:', exact_result)
qc_result = cost_numerator(test_theta)
print('qc:', qc_result)

In [ ]:
### calculate gradient ###
dev_grad_XA_oper = pl.device("default.qubit", wires=n_q+1, shots=n_shots)
@pl.qnode(dev_grad_XA_oper)
def qc_grad_XA_oper(A_oper, theta, rot_idx):
    ## ansatz unitary
    pl.Hadamard(wires=0)
    p = int((len(theta)-n_q)/(2*(n_q-1)))  # depth of ansatz
    for i in range(n_q):
        pl.RY(theta[i], wires=i+1)
        if rot_idx == i:
            pl.CRY(np.pi, wires=[0, i+1])
    # pl.Barrier(wires=[i for i in range(n_q)])
    for iter in range(p):
        for i in range(n_q//2):
            pl.CZ(wires=[2*i+1, 2*i+2])
            pl.RY(theta[n_q+2*iter*(n_q-1)+2*i], wires=2*i+1)
            if rot_idx == n_q+2*iter*(n_q-1)+2*i:
                pl.CRY(np.pi, wires=[0, 2*i+1])
            pl.RY(theta[n_q+2*iter*(n_q-1)+2*i+1], wires=2*i+2)
            if rot_idx == n_q+2*iter*(n_q-1)+2*i+1:
                pl.CRY(np.pi, wires=[0, 2*i+2])
        if n_q != 2:
            for i in range((n_q-1)//2):
                pl.CZ(wires=[2*i+2, 2*i+3])
                pl.RY(theta[n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i], wires=2*i+2)
                if rot_idx == n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i:
                    pl.CRY(np.pi, wires=[0, 2*i+2])
                pl.RY(theta[n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i+1], wires=2*i+3)
                if rot_idx == n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i+1:
                    pl.CRY(np.pi, wires=[0, 2*i+3])
    ## measurement
    pl.Hadamard(wires=0)
    if A_oper == '0'*n_q:
        return pl.counts(wires=0)
    else:
        n_cr = len(A_oper) - A_oper.count('0')
        for i in range(1, n_cr):
            pl.CNOT(wires=[n_q-i, n_q-i+1])
        pl.Hadamard(wires=n_q-n_cr+1)
        measure_list = [0] + [n_q+1-n_cr+i for i in range(n_cr)]
        return pl.counts(wires=measure_list)


def expectation_grad_XA_oper(A_oper, counts):
    if A_oper[:] == '0'*n_q:
        expectation = 2*counts.get('0', 0) - 2*counts.get('1', 0)
        expectation = expectation/n_shots
    elif A_oper.count('1') == 1:
        expectation = counts.get('00', 0) - counts.get('01', 0) - counts.get('10', 0) + counts.get('11', 0)
        expectation = -expectation/n_shots
    else:
        expectation = counts.get('001'+'0'*(A_oper.count('1')-2), 0) - counts.get('011'+'0'*(A_oper.count('1')-2), 0) \
                      - counts.get('101'+'0'*(A_oper.count('1')-2), 0) + counts.get('111'+'0'*(A_oper.count('1')-2), 0)
        expectation = -expectation/n_shots
    return expectation

In [ ]:
num_param_iter = (n_q//2 + (n_q-1)//2)*2
p = 2
num_param = n_q + num_param_iter*p
list_param = np.array([i for i in range(16)])*np.pi/8
test_theta0 = np.array([random.choice(list_param) for i in range(num_param)])
for i in range(n_q + num_param_iter*p):
    test_theta1 = test_theta0.copy()
    test_theta1[i] = test_theta1[i] + np.pi
    state = np.kron(np.array([1, 0]), np.array(get_ansatz_state(test_theta1)))/np.sqrt(2)\
            +np.kron(np.array([0, 1]), np.array(get_ansatz_state(test_theta0)))/np.sqrt(2)
    test_mat = np.kron(np.array([[0, 1], [1, 0]]), mat_1d_Dirichlet(m+1))
    exact_result = np.dot(state, test_mat @ state.transpose())
    print('exact_result:', exact_result)
    result = 0
    for bitstring in A_oper_list:
        results = qc_grad_XA_oper(A_oper=bitstring, theta=test_theta0, rot_idx=i)
        result += expectation_grad_XA_oper(A_oper=bitstring, counts=results)
    print('qc_result:', result)

In [ ]:
def expectation_grad_XA(theta, rot_idx):
    grad_XA = 0
    for A_oper in A_oper_list:
        counts = qc_grad_XA_oper(A_oper=A_oper, theta=theta, rot_idx=rot_idx)
        result = expectation_grad_XA_oper(A_oper=A_oper, counts=counts)
        grad_XA += result
    return grad_XA


dev_grad_numer_1 = pl.device("default.qubit", wires=n_q+1, shots=n_shots)
@pl.qnode(dev_grad_numer_1)
def qc_grad_numer_1_oper(theta, rot_idx):
    ## ancilla qubit
    pl.Hadamard(wires=0)
    ## ansatz unitary
    p = int((len(theta)-n_q)/(2*(n_q-1)))  # depth of ansatz
    for i in range(n_q):
        pl.RY(theta[i], wires=i+1)
        if rot_idx == i:
            pl.CRY(np.pi, wires=[0, i+1])
    # pl.Barrier(wires=[i for i in range(n_q)])
    for iter in range(p):
        for i in range(n_q//2):
            pl.CZ(wires=[2*i+1, 2*i+2])
            pl.RY(theta[n_q+2*iter*(n_q-1)+2*i], wires=2*i+1)
            if rot_idx == n_q+2*iter*(n_q-1)+2*i:
                pl.CRY(np.pi, wires=[0, 2*i+1])
            pl.RY(theta[n_q+2*iter*(n_q-1)+2*i+1], wires=2*i+2)
            if rot_idx == n_q+2*iter*(n_q-1)+2*i+1:
                pl.CRY(np.pi, wires=[0, 2*i+2])
        if n_q != 2:
            for i in range((n_q-1)//2):
                pl.CZ(wires=[2*i+2, 2*i+3])
                pl.RY(theta[n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i], wires=2*i+2)
                if rot_idx == n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i:
                    pl.CRY(np.pi, wires=[0, 2*i+2])
                pl.RY(theta[n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i+1], wires=2*i+3)
                if rot_idx == n_q+2*iter*(n_q-1)+2*(n_q//2)+2*i+1:
                    pl.CRY(np.pi, wires=[0, 2*i+3])
    ## measurement
    pl.Hadamard(wires=0)
    ## measurement
    ### problem 1 - RHS: constant function ###
    # for i in range(n_q):
    #     pl.Hadamard(wires=i+1)
    ### problem 2 - RHS: step function ###
    pl.PauliZ(wires=1)
    for i in range(n_q):
        pl.Hadamard(wires=i+1)
    return pl.counts(wires=[i for i in range(n_q+1)])

### estimate grad_numer_1 ###
def cost_grad_numer_1(theta, rot_idx):
    counts = qc_grad_numer_1_oper(theta=theta, rot_idx=rot_idx)
    cost = counts.get('0'*(n_q+1), 0)/n_shots
    return cost

In [ ]:
def gradient(theta):
    start = time.time()
    def get_gradient(i):
        theta0 = theta
        theta1 = theta.copy()
        theta1[i] = theta1[i] + np.pi
        val_numer_1 = (cost_grad_numer_1(theta=theta0, rot_idx=i)*4-cost_numerator(theta1)-numer_cost)/2
        val_numer_2 = expectation_grad_XA(theta=theta0, rot_idx=i)
        grad = (-val_numer_1/denom_cost + numer_cost*val_numer_2/(denom_cost**2))/2
        return grad
    grad_list = Parallel(n_jobs=-1, prefer='processes')(delayed(get_gradient)(i) for i in range(len(theta)))
    end = time.time()
    print('gradient time:', end-start)
    return grad_list

In [ ]:
def vqa_optimize(init_param, optimizer):
    def get_cost():
        def execute_circ(theta):
            start = time.time()
            global denom_cost, numer_cost
            denom_cost = cost_denominator(theta)
            numer_cost = cost_numerator(theta)
            cost = -(1/2)*numer_cost/denom_cost
            end = time.time()
            print('cost:', cost)
            print('cost time:', end - start)
            return cost
        return execute_circ
    cost_ftn = get_cost()
    if optimizer == 'BFGS':
        res = minimize(cost_ftn, init_param, method=optimizer, jac=gradient)
    elif optimizer == 'L-BFGS-B':
        res = minimize(cost_ftn, init_param, method=optimizer, jac=gradient)
    elif optimizer == 'COBYLA':
        res = minimize(cost_ftn, init_param, method=optimizer)
    return res, res.x

In [ ]:
from scipy.optimize import minimize

p = 3
for i in range(1):
    num_param = n_q + 2*p*(n_q-1)
    # ==========
    list_param = np.array([i for i in range(32)])/8
    param_list = np.array([random.choice(list_param) for i in range(num_param)])
    init_param = param_list*np.pi
    # ==========
    optimizer = 'L-BFGS-B'
    result = vqa_optimize(init_param=init_param, optimizer=optimizer)
    print(result[0])
    qa_state = get_ansatz_state(result[1])
    qa_state = np.array([qa_state[i].real for i in range(2**n_q)])
    print(qa_state)
    fidel_test = abs(np.dot(qa_state, normalized_sol))
    print('fidel:', fidel_test)
    qa_norm = np.sqrt(numer_cost)/denom_cost
    norm_difference_ratio = abs((qa_norm - exact_norm)/exact_norm)
    print('norm difference ratio:', norm_difference_ratio)